# MRI pre-processing

## Pre-processing pipeline figure here

## Inter subject co-registration

Present 3D

Presen the idea with two brain images: 1) far from each other, 2) on top of each other - a smallest possible region of interest - voxel

This is typically accomplished in two phases: 1) linear, and 2) non-linear co-registration. Linear makes global changes; non-linear also local changes.

Linear co-registration
* rigid - 6 parameters, 3 translation, 3 rotation (figure)
* affine - 9 or 12 parameters (figure)

All of the linear transformations are global and can be descrbed with a  4x4 transform matrix,

$$
T =
\begin{pmatrix}
   0.1 & 0   & 0   & -5   \\
   0   & 0.1 & 0   & -6.7 \\
   0   & 0   & 0.1 & -10  \\
   0   & 0   & 0   & 1
\end{pmatrix}
$$

This is a matrix describing a space with isotropic voxel size (of 0.1 units) and a translation of -5 -6.7 -10 units (usually in x,y,z coordinates).

* translation (ok)

$$
T_{trans} =
\begin{pmatrix}
   1 & 0 & 30 \\
   0 & 1 & 30 \\
   0 & 0 & 1  
\end{pmatrix}
$$

* scaling (ok)

$$
T_{scaling} =
\begin{pmatrix}
   1.3 & 0   & 10 \\
   0   & 0.8 & 30 \\
   0   & 0   & 1  
\end{pmatrix}
$$


* rotation (ok)


$$
T_{rot} =
\begin{pmatrix}
   \cos(\theta) & -\sin(\theta) & 30 \\
   \sin(\theta) &  \cos(\theta) & 30 \\
   0            & 0             & 1  
\end{pmatrix}
=
\begin{pmatrix}
   0.980 & -0.199 & 40 \\
   0.199 &  0.980 & 20 \\
   0     &  0     & 1  
\end{pmatrix}
$$

when $$ \theta = .2 $$

* shear fig
$$
T_{shear} =
\begin{pmatrix}
   1   & 0.1 & 10 \\
   0.1 & 1   & 30 \\
   0   & 0   & 1  
\end{pmatrix}
$$

* perspective (which is 0 in our case always?) ok
$$
T_{scaling} =
\begin{pmatrix}
   1     & 0 & 10 \\
   0     & 1 & 30 \\
   0.001 & 0 & 1  
\end{pmatrix}
$$

* application by a matrix-vector multiplication

\begin{gather}
  v_{old} =
  \begin{pmatrix}
      x\\
      y\\
      z
  \end{pmatrix}
  \\
  v_{new} = T v_o
\end{gather}

* chaining of two transforms by matrix-matrix multiplication fig?

$$ T_t = T_2 T_1 $$

Talk about interpolation too

Non-linear co-registration
* talk about the liberties added compared to the linear case
* ultimately everything fits everything -> we need constraints
* in our case, one such constraint could be nearby objects should remain nearby objects after transform

Metrics

* cross correlation - same to same - show images and scatter plots
    * show two brain images partially on top of each other and scatter plot
    * show them well on each other and scatter plot
    
$$
r =\frac{n \sum{xy} - (\sum{x})(\sum{y})}{\sqrt{\left(n \sum{x^2}-(\sum{x})^2\right)\left(n \sum{y^2}-(\sum{y})^2\right)}}
$$

* mutual information - different to different
    * show images, talk about common information
    * show scatterplot -> no correlation, show joint information scatter -> correlation

$$
\mathrm {I}(X;Y) = \sum_{y \in Y}{\sum_{x \in X}{\mathrm {P}_{(X,Y)}(x,y) \log{\frac{\mathrm {P}_{(X,Y)}(x,y)}{\mathrm {P}_X(x) \mathrm {P}_Y(y)}}}}  
$$

$$
\mathrm {P}_{X,Y}(x,y)=\mathrm {P} (X=x\ \mathrm {and} \ Y=y)
$$	


* how does this simplify to intra-subject?
     * only rigid needed?



Additional properties of the co-registration that we might want
* also would like to perform transformations backwards without computing the transforms again
    * 4x4 matrix transpose is the inverse
        * non-linear mapping is not that easy to invert? -> some mechanism needed; estimation of both directions simultaneously?


## (time-)series preprocessing

Talk about how this is just a more simple application of the previous tools.
    Same brain, contrast etc: rigid co-registration except for susceptibility-induced artefacts
    or distance-related field inhomogeneities
    
fMRI & dMRI
* series of 3D volumes
    * fMRI: blood oxygen-induced changes in the time-series
    * dMRI: diffusion sensitizing gradient-induced differences in the volumes
        * show in video
* if the series contain subject motion, voxel (time-)series are not reliable
    * motion correction (with references to co-reg; rigid w CC-type metric)
        * rigid co-reg if no acquisition-induced distortions
            * video of pre and post
        * fMRI specific: slice-timing
            * 2D acquisitions - if slices are 
        * dMRI specific: eddy current correction
            * gradient changes induce eddy currents that can be corrected with specialiced algorigthms
        * any way to show the distortions?

## Spatial de-noising

dti enemmän

## Smoothing

registration, jotkin algoritmit toimivat paremmin
multi-subject: poistaa pieniä variaatioita
tilastollinen inferenssi: smooth -> datasta enemmän gaussista

## Show the whole pre-processing pipeline here again

How does the process look like (when using the tools presented earlier)

An example code in Snakemake formalism performing slice timing and motion correction to a dataset ../derivatives/{dataset}_EPI_ms.nii.gz, where {dataset} stands for any string. Slice timing is performed using FSL tool slicetimer and motion correction with ANTs tool antsMotionCorr.

```python
rule motion_correction:
    input:
        data="../derivatives/{dataset}_EPI_ms_st.nii.gz",
        reference="../derivatives/{dataset}_EPI_ms_st_reference.nii.gz"
    output:
        "../derivatives/{dataset}_EPI_ms_st_mc.nii.gz"
    params:
        prefix="../derivatives/{dataset}_EPI_st"
    shell:
        "antsMotionCorr --dimensionality 3 --output [{params.prefix}_mc_,{output},{params.prefix}_ref.nii.gz] --transform Rigid[0.1] --metric GC[{input.reference},{input.data},1,32,Regular,0.2] --iterations 50x50 --useFixedReferenceImage 1 --useScalesEstimator 1 --smoothingSigmas 2x0 --shrinkFactors 2x1"

rule slice_timing:
    input:
        "../derivatives/{dataset}_EPI_ms.nii.gz"
    output:
        "../derivatives/{dataset}_EPI_ms_st.nii.gz"
    shell:
        "slicetimer -i {input} -o {output} -r 1 --odd"
```

* show with images & videos
* voxel time-series before and after the process fig
* työkaluja löytyy
    * spesifisiä tai yleisiä
    * osassa graafiset käyttöliittymät
    * lista tähän

## Parametric maps from the data

* DTI fitting -> FA, MD, ...
    * tensor model - ellipsoid
    * eigen decomposition -> eigenvalues and vectors
    * eigenvalues -> parametric maps FA, MD show these

* T1 T1 - rekot
